In [7]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import sys
import os
from pathlib import Path
import cv2  # Import OpenCV

# Add the scripts folder to the system path so we can import Db_connectivity
sys.path.append(os.path.abspath('../scripts'))

from Db_connectivity import connect_to_db  # Import the connection function from your module

In [8]:
# Define the directory containing images
#images_dir = 'C:/Users/user/Desktop/Github/Data_Warehouse_ForEthioMedical/photos'  # Replace with actual directory path

# Define the database connection
DATABASE_NAME = 'ethiomedical_info'
conn = connect_to_db(DATABASE_NAME)



Connection to PostgreSQL database established successfully.


In [10]:
# Function to load detections from the database
def load_detections_from_db(image_name):
    conn = connect_to_db(DATABASE_NAME)
    
    # Adjust the query to fit your database structure
    detections_query = f"SELECT * FROM yolo_detections WHERE class = '{image_name}'"  # Use class for filtering
    detections = pd.read_sql(detections_query, conn)

    # Convert columns to numeric
    detections['x_min'] = pd.to_numeric(detections['x_min'], errors='coerce')
    detections['y_min'] = pd.to_numeric(detections['y_min'], errors='coerce')
    detections['x_max'] = pd.to_numeric(detections['x_max'], errors='coerce')
    detections['y_max'] = pd.to_numeric(detections['y_max'], errors='coerce')

    conn.close()
    return detections

In [11]:
# Function to visualize detections
def visualize_detections(image_path, detections):
    img = cv2.imread(str(image_path))
    for _, row in detections.iterrows():
        x_min = int(round(row['x_min']))
        y_min = int(round(row['y_min']))
        x_max = int(round(row['x_max']))
        y_max = int(round(row['y_max']))
        confidence = row['confidence'] / 100  # Convert back to proportion
        label = f"{row['class']} {confidence:.2f}"
        
        # Draw the bounding box and label
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        cv2.putText(img, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Display the image
    cv2.imshow("Detections", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [12]:
# Main function to load and visualize detections for multiple images
def main():
    # List of image names corresponding to their class (update with your actual class names)
    image_names = [75,14,74,27,56]
    
    # Iterate through each image and visualize detections
    for image_name in image_names:
        detections = load_detections_from_db(image_name)

        if not detections.empty:
            # Set the image path manually (change this to your specific image path)
            image_path = Path(f'C:/Users/user/Desktop/Github/Data_Warehouse_ForEthioMedical/photos/{image_name}.jpg')
            visualize_detections(image_path, detections)
        else:
            print(f"No detections found in the database for {image_name}.")

if __name__ == "__main__":
    main()

Connection to PostgreSQL database established successfully.


C:\Users\user\AppData\Local\Temp\ipykernel_20740\1943943378.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  detections = pd.read_sql(detections_query, conn)


Connection to PostgreSQL database established successfully.
Connection to PostgreSQL database established successfully.
Connection to PostgreSQL database established successfully.
Connection to PostgreSQL database established successfully.
